### Поправки на множественную проверку при корреляционном анализе
Анализировать будем уже знакомый dataset ```Foodmart Product Sales```

In [29]:
import pandas as pd

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

### Загрузка данных

In [30]:
products = pd.read_csv('foodmart_products.csv', sep='\t')
sales = pd.read_csv('foodmart_sales.csv', sep='\t', parse_dates=[2])

product_sales = sales.merge(products[['product_id', 'product_name']], on='product_id', how='inner')
product_sales.head()

,product_id,store_id,date,sales,product_name
0,4,6,1997-01-01,4,Washington Cream Soda
1,4,7,1997-01-05,3,Washington Cream Soda
2,4,6,1997-01-06,2,Washington Cream Soda
3,4,17,1997-01-11,2,Washington Cream Soda
4,4,24,1997-01-11,2,Washington Cream Soda


### Таблица корреляция
Как и в прошлый раз получим таблицу с корреляциями продуктов между собой

In [31]:
product_corr_table = pd.pivot_table(product_sales,
                                    values='sales',
                                    index=['date', 'store_id'],
                                    columns='product_name',
                                    aggfunc=lambda x: x,
                                    fill_value=0)

product_corr_table.head()

product_name         ADJ Rosy Sunglasses  Akron City Map  \
date       store_id                                        
1997-01-01 6                           0               0   
           14                          0               0   
1997-01-02 11                          0               0   
           23                          0               0   
1997-01-03 7                           0               0   

product_name         Akron Eyeglass Screwdriver  American Beef Bologna  \
date       store_id                                                      
1997-01-01 6                                  0                      0   
           14                                 0                      0   
1997-01-02 11                                 0                      0   
           23                                 0                      0   
1997-01-03 7                                  0                      0   

product_name         American Chicken Hot Dogs  American Cole Slaw  \
date       store_id                                                  
1997-01-01 6                                 0                   0   
           14                                0                   0   
1997-01-02 11                                0                   0   
           23                                0                   0   
1997-01-03 7                                 0                   0   

product_name         American Corned Beef  American Foot-Long Hot Dogs  \
date       store_id                                                      
1997-01-01 6                            0                            0   
           14                           0                            0   
1997-01-02 11                           0                            0   
           23                           0                            0   
1997-01-03 7                            0                            0   

product_name         American Low Fat Bologna  American Low Fat Cole Slaw  \
date       store_id                                                         
1997-01-01 6                                0                           0   
           14                               0                           0   
1997-01-02 11                               0                           0   
           23                               0                           0   
1997-01-03 7                                0                           0   

product_name         ...  Washington Apple Juice  Washington Berry Juice  \
date       store_id  ...                                                   
1997-01-01 6         ...                       0                       0   
           14        ...                       0                       0   
1997-01-02 11        ...                       0                       0   
           23        ...                       0                       0   
1997-01-03 7         ...                       0                       4   

product_name         Washington Cola  Washington Cranberry Juice  \
date       store_id                                                
1997-01-01 6                       0                           0   
           14                      0                           0   
1997-01-02 11                      0                           0   
           23                      0                           0   
1997-01-03 7                       0                           0   

product_name         Washington Cream Soda  Washington Diet Cola  \
date       store_id                                                
1997-01-01 6                             4                     0   
           14                            0                     0   
1997-01-02 11                            0                     0   
           23                            0                     0   
1997-01-03 7                             0                     0   

product_name         Wa

Мы хотим проверить, есть ли корреляция между покупками пары продуктов. При этом мы хотим не только посчитать значение корреляции, а также проверить гипотезу о том, что корреляция присутствует, против альтернативы что ее нет (т.е. получить достигаемые уровни значимости).

Для того чтобы это получить, нам понадобится функция ```pearsonr```, это функция модуля ```stats``` библиотеки ```SciPy```.С этой целью мы пройдемся циклом по парам продуктов и для каждой пары рассчитаем корреляцию и значение p-value, и дальше соберем всё это в DataFrame со столбцами пары продуктов, их корреляции и значения p-value.

Функция ```pearsonr``` возвращает корреляцию и p-value 

In [32]:
%%time 
corr_values = []

# Считаем попарные корреляции между продуктами
for i, prod_a in enumerate(product_corr_table.columns):
    for j, prod_b in enumerate(product_corr_table.columns):
        # Корреляции продуктов с собой и тех которые уже были пропускаем
        if i >= j:
            continue
        # Считаем корреляции
        corr, p_val = pearsonr(product_corr_table[prod_a], product_corr_table[prod_b])
        corr_values.append([prod_a, prod_b, corr, p_val])

Wall time: 1min 41s


In [34]:
# Оформим DataFrame
prod_corr_df = pd.DataFrame(corr_values, columns=['prod_a', 'prod_b', 'corr', 'p_value'])
prod_corr_df.head()

,prod_a,prod_b,corr,p_value
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484


Получили корреляции пары продуктов и их уровни значимости. Сейчас мы будем применять множественную проверку гипотез, т.е мы будем сравнивать все наши объекты (таких 1.2M) с некотрым уровнем значимости и смотреть сколько гипотез принимается/отвергается.

H_0: корреляции нет. Посмотрим сколько таких гипотез отвергается без поправки на множественное сравнение.

In [35]:
(prod_corr_df['p_value'] <= 0.05).value_counts()

False    982453
True     232008
Name: p_value, dtype: int64

Около 200к гипотез отвергается, неплохо. Теперь внесем поправку на множественное сравнение.

### Поправки на множественное сравнение гипотез
### Метод Холма
Для этого используем ```multipletests```. Метод возвращает:
- Отвергается ли нулевая гипотеза для конкретного наблюдения
- Скорректированное значение p-value

In [38]:
# Рассчитываем с учетом на множественное сравнение
is_rejected, p_corrected, a1, a2 = multipletests(prod_corr_df['p_value'], alpha=0.05, method='holm')

# Добавляем в основной DF
prod_corr_df['p_corrected'] = p_corrected
prod_corr_df['is_rejected'] = is_rejected
prod_corr_df.head()

,prod_a,prod_b,corr,p_value,p_corrected,is_rejected
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,1.0,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,1.0,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,1.0,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,1.0,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,1.0,False


Теперь видим, что p-value скорректировался. Узнаем сколько теперь гипотез отвергается

In [39]:
prod_corr_df['is_rejected'].value_counts()

False    1212733
True        1728
Name: is_rejected, dtype: int64

Число отвергаемых гипотез стало существенно меньше. Взглянем на значимые корреляции и самые сильные.

In [40]:
prod_corr_df[prod_corr_df['is_rejected'] == True].sort_values('corr', ascending=False).head()

,prod_a,prod_b,corr,p_value,p_corrected,is_rejected
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,3.265443e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,2.288034e-12,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,5.677558e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,1.966350e-11,True


### Метод Бенджамини-Хохберга
В этом методе мы будем ограничивать среднюю вероятность ошибок

In [41]:
# Рассчитываем с учетом на множественное сравнение
is_rejected, p_corrected, a1, a2 = multipletests(prod_corr_df['p_value'], alpha=0.05, method='fdr_bh')

# Добавляем в основной DF
prod_corr_df['p_corrected'] = p_corrected
prod_corr_df['is_rejected'] = is_rejected
prod_corr_df.head()

,prod_a,prod_b,corr,p_value,p_corrected,is_rejected
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,0.203716,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,0.956078,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,0.630699,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,0.600790,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,0.541916,False


In [42]:
prod_corr_df['is_rejected'].value_counts()

False    1138407
True       76054
Name: is_rejected, dtype: int64

In [43]:
prod_corr_df[prod_corr_df['is_rejected'] == True].sort_values('corr', ascending=False).head()

,prod_a,prod_b,corr,p_value,p_corrected,is_rejected
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,1.632723e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,7.626793e-13,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,1.419393e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,3.932713e-12,True


In [52]:
from scipy import stats
from itertools import combinations

### Задание из теста 
**1)** Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:

In [48]:
data = pd.read_csv('AUCs.txt', sep='\t')
data.columns = ['dataset', 'C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf']
data.head()

,dataset,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо. 

Выборки зависимые, необходимо использовать критерий знаковых рангов Вилкоксона

In [62]:
# Для начала создадим результирующий df
modesl_df = pd.DataFrame(columns=['Model_A', 'Model_B', 'Wilcoxon_Stat', 'p_value'])

# Получаем все возможные комбинации наших столбцов (моделей) при помощи модуля combinations
for indx, models in enumerate(combinations(data.columns[1:].values, 2)):
    modesl_df.loc[indx, 'Model_A'], modesl_df.loc[indx, 'Model_B'] = models[0], models[1]
    modesl_df.loc[indx, 'Wilcoxon_Stat'], modesl_df.loc[indx, 'p_value'] = stats.wilcoxon(data.loc[:, models[0]], data.loc[:, models[1]])
    
modesl_df

c:\users\vlad\appdata\local\programs\python\python37\lib\site-packages\scipy\stats\morestats.py:2957: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,Model_A,Model_B,Wilcoxon_Stat,p_value
0,C4.5,C4.5+m,6.5,0.0107571
1,C4.5,C4.5+cf,43,0.861262
2,C4.5,C4.5+m+cf,11,0.0159064
3,C4.5+m,C4.5+cf,17,0.0463327
4,C4.5+m,C4.5+m+cf,22,0.327826
5,C4.5+cf,C4.5+m+cf,10,0.0229091


Критерий показал, что есть как значимые так и незначимые отличия. Наиболее значимые различия между C4.5 и C4.5+m

**2)** Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [63]:
(modesl_df['p_value'] <= 0.05).value_counts()

True     4
False    2
Name: p_value, dtype: int64

Мы обнаружили 4 статистически значимых различия

**3)** 
Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

Это параметр m

**4)** Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [66]:
# Рассчитываем с учетом на множественное сравнение
is_rejected, p_corrected, a1, a2 = multipletests(modesl_df['p_value'], alpha=0.05, method='holm')

modesl_df['p_corrected'] = p_corrected
modesl_df['is_rejected'] = is_rejected

modesl_df['is_rejected'].value_counts()

False    6
Name: is_rejected, dtype: int64

Учитывая поправку на множественное сравнение мы теперь не можем отвергнуть ни одну из гипотез.

**5)** Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [67]:
is_rejected, p_corrected, a1, a2 = multipletests(modesl_df['p_value'], alpha=0.05, method='fdr_bh')

modesl_df['p_corrected'] = p_corrected
modesl_df['is_rejected'] = is_rejected

modesl_df['is_rejected'].value_counts()

True     3
False    3
Name: is_rejected, dtype: int64

Данный метод позволяет на отвергнуть 3 гипотезы. Однако не стоит забывать, что метод Бенджамини-Хохберга требует независимости групп, что в нашем случае неверно. Выборки зависимые. Мы не можем доверять данному критерию.

Выходит, что классификаторы значимо между собой не отличаются

**6)** Мы подозреваем, что в проведённом с C4.5 эксперименте на самом деле классификаторы сильнее отличаются друг от друга, просто нам не удалось это заметить. Что можно сделать, чтобы увеличить вероятность обнаружения различий, если они действительно существуют?
- взять больше наблюдений (датасетов)